# Compression algorithms

### Huffman:

In [24]:
def huffman(data):
    ...

### LZ77 / LZ78

In [25]:
def lz77(data):
    ...

#or

def lz78(data):
    ...

### LZW

In [ ]:
def lzw(data):
    ...

### Deflate

In [28]:
def deflate(data):
    ...

### Other

In [29]:
def other(data):
    ...

In [33]:
lzw = LZW()
'/home/gllekk/all_the_code/default/DISCRETE/compression_research/img_test_1.png'
'/home/gllekk/all_the_code/default/DISCRETE/compression_research/img_test_1_after.lzw'
def encoding(path:str, compress_algorithm:object):
    with open(path, 'rb') as file:
        image = file.read()
    encoded_data, encoded_dict = compress_algorithm.encode(image)
    file_type = path[len(path)-3:]
    with open((path:=path[:-3]+compress_algorithm.name.lower()), 'wb') as file:
        for value in encoded_data:
            file.write(value.to_bytes(4, byteorder='big'))
    return path, encoded_dict, file_type, compress_algorithm

def decoding(path:str, start_dict:dict, f_type:str, compress_algorithm:object):
    with open(path, 'rb') as file:
        encoded_data = []
        while (byte := file.read(4)):
            encoded_data.append(int.from_bytes(byte, byteorder='big'))
    decoded = compress_algorithm.decode(encoded_data, start_dict)

    with open(path[:-4]+'_decoded.'+f_type, 'wb') as file:
        file.write(decoded)

enc = encoding('/home/gllekk/all_the_code/default/DISCRETE/compression_research/img_test_1.png', lzw)
# enc = encoding('/home/gllekk/all_the_code/default/DISCRETE/compression_research/lorem.txt', 'lzw', lzw)
# enc = encoding('/home/gllekk/all_the_code/default/DISCRETE/compression_research/mixkit-fast-rocket-whoosh-1714.wav', lzw)
dec = decoding(enc[0], enc[1], enc[2], enc[3])